# Advent of Code 2020

In [ ]:
import numpy as np
import pandas as pd
import itertools
import re
import functools

## Day 1
### part 1

In [ ]:
with open('1.txt') as numberfile:
    numbers = np.array(numberfile.read().split(), dtype=int)

In [ ]:
for i in numbers:
    if (product := numbers[numbers + i == 2020] * i).size > 0:
        print(product)

### part 2

In [ ]:
for i, n in enumerate(numbers):
    for m in numbers[i:]:
        if (product := numbers[numbers + n + m == 2020] * n * m).size > 0:
            print(product)

## Day 2
### part 1

In [ ]:
passwords = pd.read_csv('2.txt', sep=' |-|: ', 
                        names=['min_counts', 'max_counts', 'character', 'password'], 
                        engine='python')

In [ ]:
valid_passwords = 0
for entry in passwords.itertuples():
    if entry.min_counts <= entry.password.count(entry.character) <= entry.max_counts:
        valid_passwords += 1
print(valid_passwords)

### part 2

In [ ]:
passwords.rename(columns={'min_counts': 'first_index', 'max_counts': 'last_index'}, inplace=True)
valid_passwords = 0
for entry in passwords.itertuples():
    if (entry.password[entry.first_index-1] == entry.character) ^ (entry.password[entry.last_index-1] == entry.character):
        valid_passwords += 1
print(valid_passwords)

## Day 3
### part 1

In [ ]:
tree_counter = 0
index = 0
slope = 3
with open('3.txt') as tree_pattern_file:
    for row in tree_pattern_file:
        row = row[:-1]
        tree_counter += (row[index] == '#')
        index = (index + slope) % (len(row))
print(tree_counter)

### part 2

In [ ]:
slope = np.arange(1, 8, 2, dtype='int64')
index = np.zeros_like(slope)
tree_counter = np.zeros_like(slope)

with open('3.txt') as tree_pattern_file:
    for row in tree_pattern_file:
        row = row[:-1]
        tree_counter += [row[i] == '#' for i in index]
        index = (index + slope) % (len(row))

# Special case
s_slope = 1
s_index = 0
s_tree_counter = 0
with open('3.txt') as tree_pattern_file:
    for row in itertools.islice(tree_pattern_file, 0, None, 2):
        row = row[:-1]
        s_tree_counter += (row[s_index] == '#')
        s_index = (s_index + s_slope) % (len(row))
print(tree_counter.prod() * s_tree_counter)

## Day 4
### part 1

In [ ]:
with open('4.txt') as passport_file:
    passports = passport_file.read()
pass_port_keys = ['byr','iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
valid_passports = 0
for p_info in passports.split('\n\n'):
    valid_passports += all(key in p_info for key in pass_port_keys)
print(valid_passports)

### part 2

In [ ]:
def create_dict(passport_info):
    for info in passport_info.split():
        yield info.split(':')

In [ ]:
with open('4.txt') as passport_file:
    passports = passport_file.read()
passport_keys = ['byr','iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid', 'cid']
valid_passports = 0
for p_info in passports.split('\n\n'):
    p_info_dict = {key: value for key, value in create_dict(p_info)}
    # Guard clauses for input validation
    try:
        if not (set(p_info_dict.keys()) | set(('cid',))) == set(passport_keys):
            continue
        elif not 1920 <= int(p_info_dict['byr']) <= 2002:
            continue
        elif not 2010 <= int(p_info_dict['iyr']) <= 2020:
            continue
        elif not 2020 <= int(p_info_dict['eyr']) <= 2030:
            continue
        elif not ((('cm' in p_info_dict['hgt']) and (150 <= int(p_info_dict['hgt'][:-2]) <= 193))
                  or ('in' in p_info_dict['hgt'] and (59 <= int(p_info_dict['hgt'][:-2]) <= 76))):
            continue
        elif re.fullmatch('#(?:[0-9a-f]{6})', p_info_dict['hcl']) is None:
            continue
        elif p_info_dict['ecl'] not in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
            continue
        elif not (p_info_dict['pid'].isnumeric() and len(p_info_dict['pid']) == 9):
            continue
    except Exception as e:
        print(p_info_dict, e)
    valid_passports += 1
    
print(valid_passports)

## Day 5
### part 1

In [ ]:
a = 'i'
a.replace('i', '0')
a

In [ ]:
with open('5.txt') as seat_file:
    seats = seat_file.read()
bin_seats = seats.replace('F', '0')
bin_seats = bin_seats.replace('B', '1')
bin_seats = bin_seats.replace('L', '0')
bin_seats = bin_seats.replace('R', '1')
rows = np.array([int(s[:-3], base=2) for s in bin_seats.split('\n')], dtype=int)
columns = np.array([int(s[-3:], base=2) for s in bin_seats.split('\n')], dtype=int)
seat_id = np.array([int(s, base=2) for s in bin_seats.split('\n')])

print((rows*8+columns).max(), seat_id.max()) # Both should produce same output

### part 2

In [ ]:
seat_id.sort()
seat_id[np.append(np.diff(seat_id) != 1, False)] + 1

## Day 6
### part 1

In [ ]:
sum(len(set(group_answers.replace('\n', ''))) for group_answers in open('6.txt').read().split('\n\n'))

In [ ]:
sum(len(functools.reduce(set.__and__, (set(individual_answer) for individual_answer in group.split('\n'))))
    for group in open('6.txt').read().split('\n\n'))

## Day 7
### part 1

In [ ]:
def count_outer_bags(bag_name):
    outer_bag_index = bags['inner'].str.contains(bag_name)
    if outer_bag_index.sum() == 0:
        return set()
    return set(bags['outer'][outer_bag_index]) | functools.reduce(set.__or__, (count_outer_bags(bag) for bag in bags['outer'][outer_bag_index]))

In [ ]:
bags = pd.read_csv('7.txt', sep='bags contain ', names=['outer', 'inner'], engine='python')
len(count_outer_bags('shiny gold'))

### part 2

In [ ]:
bag_rules = {}
with open('7.txt') as bag_rule_file:
    for bag_rule in bag_rule_file:
        bag_rule = bag_rule.strip('\n')
        bag_name = bag_rule[:bag_rule.index(' bags contain ')]
        if bag_rule.endswith('contain no other bags.'):
            bag_rules[bag_name] = []
        else:
            # I barely know what this line does but it works
            inner_bags = re.split(' bags.| bags, | bag.| bag, ', bag_rule[bag_rule.index('s contain ')+len('s contain'):])
            inner_bags.remove('')
            bag_rules[bag_name] = [{'bag name': inner_bag[3:], 'count': int(inner_bag[1])} for inner_bag in inner_bags]

In [ ]:
@functools.cache
def count_inner_bags(bag_name):
    if bag_rules[bag_name]:
        return (sum(inner_bag['count']*count_inner_bags(inner_bag['bag name']) 
                   for inner_bag in bag_rules[bag_name]) + 1)
    else:
        return 1

In [ ]:
count_inner_bags('shiny gold')-1

## Day 8
### part 1

In [ ]:
instructions = pd.read_csv('8.txt', sep=' ', names=['instruction', 'value'])
visited_instructions = []
i = 0
acc = 0
while i not in visited_instructions:
    visited_instructions += [i]
    if instructions.loc[i, 'instruction'] == 'acc':
        acc += instructions.loc[i, 'value']
        i += 1
    elif instructions.loc[i, 'instruction'] == 'jmp':
        i += instructions.loc[i, 'value']
    else:
        i += 1
print(acc)

### part 2

In [ ]:
def do_instructions(instructions):
    visited_instructions = []
    i = 0
    acc = 0
    
    while (i not in visited_instructions) and (i <= instructions.index[-1]):
        visited_instructions += [i]
        if instructions.loc[i, 'instruction'] == 'acc':
            acc += instructions.loc[i, 'value']
            i += 1
            continue
        elif instructions.loc[i, 'instruction'] == 'jmp':
            i += instructions.loc[i, 'value']    
        else:
            i += 1
    return i, acc

In [ ]:
for change_index in instructions.query("instruction == 'jmp' or instruction == 'nop'").index:
    modified_instructions = instructions.copy()
    modified_instructions.loc[change_index, 'instruction'] = 'jmp' if modified_instructions.loc[change_index, 'instruction'] == 'nop' else 'nop'
    i, acc = do_instructions(modified_instructions)
    if i > modified_instructions.index[-1]:
        print(acc)
        break

## Day 9
### part 1

In [ ]:
with open('9.txt') as 